In [ ]:
using System.Net;
using System.Net.Http;
using System.Net.NetworkInformation;
using System.Text.Json;

In [ ]:
public class BinanceApp
{
    private const string BASE_URL = "https://api.binance.com";

    private bool isRunning = true;

    private HttpClient httpClient = null;

    private BinancePingService pingService = null;
    private BinanceTimeService timeService = null;

    public BinanceApp()
    {
        this.httpClient = new HttpClient();

        this.pingService = new BinancePingService(this.httpClient, BASE_URL, "/api/v3/ping");
        this.timeService = new BinanceTimeService(this.httpClient, BASE_URL, "/api/v3/time");
    }

    public void StartApp()
    {
        if (!this.TryPingBaseUrl() || !this.TryApiPingRequest()) {
            Console.WriteLine("Failed to ping app.");
            return;
        }

        this.isRunning = true;
    }

    public void UpdateApp()
    {
        this.TryGetServerTime();
    }

    private bool TryGetServerTime()
    {
        Task<DateTime> serverTimeTask = Task.Run<DateTime>(async () => await this.timeService.GetServerTime());

        if (serverTimeTask.Result != null) {
            // Console.WriteLine("Server time: " + serverTimeTask.Result.ToString());
            return true;
        }

        return false;
    }

    /// Ping ap ping path to check if the app is running.
    private bool TryApiPingRequest()
    {
        Task<bool> updateTask = Task.Run<bool>(async () => await this.pingService.TryApiPingRequest());
        return updateTask.Result;
    }

    /// Ping the base url to check if the app is running.
    private bool TryPingBaseUrl()
    {
        Task<bool> updateTask = Task.Run<bool>(async () => await this.pingService.TryPingBaseUrl());
        return updateTask.Result;
    }
}

public class BinancePingService
{
    private const string API_PING_METHOD = "GET";

    private string apiBaseUrl = string.Empty;
    private string apiPingPath = string.Empty;

    private HttpClient httpClient = null;

    public BinancePingService(HttpClient httpClient, string apiBaseUrl, string apiPingPath)
    {
        this.httpClient = httpClient;
        this.apiBaseUrl = apiBaseUrl;
        this.apiPingPath = apiPingPath;
    }

    public async Task<bool> TryApiPingRequest()
    {
        try {
            string endPoint = $"{this.apiBaseUrl}{this.apiPingPath}";
            HttpResponseMessage apiPingresult = await this.httpClient.GetAsync(endPoint);

            HttpResponseMessageHandler.LogResponse((apiPingresult));

            return apiPingresult.StatusCode == HttpStatusCode.OK;
        }
        catch (Exception e) {
            ExceptionHandler.LogException(e);
        }

        return false;
    }

    public async Task<bool> TryPingBaseUrl()
    {
        try {
            Ping ping = new Ping();
            // Ping url must not contain {https://}.
            PingReply pingResult = await ping.SendPingAsync(this.apiBaseUrl.Replace("https://", ""));

            PingReplyHandler.LogReply(pingResult);
            
            return pingResult.Status == IPStatus.Success;
        } catch (Exception e) {
            ExceptionHandler.LogException(e);
        }

        return false;
    }
}

public class BinanceTimeService
{
    private const string API_TIME_METHOD = "GET";

    private string apiBaseUrl = string.Empty;
    private string apiTimePath = string.Empty;

    private DateTime invalidTime = new DateTime(1970, 1, 1, 0, 0, 0, DateTimeKind.Utc);

    private HttpClient httpClient = null;

    public BinanceTimeService(HttpClient httpClient, string apiBaseUrl, string apiTimePath)
    {
        this.httpClient = httpClient;
        this.apiBaseUrl = apiBaseUrl;
        this.apiTimePath = apiTimePath;
    }

    public async Task<DateTime> GetServerTime()
    {
        try {
            string endPoint = $"{this.apiBaseUrl}{this.apiTimePath}";
            HttpResponseMessage apiTimeResult = await this.httpClient.GetAsync(endPoint);

            HttpResponseMessageHandler.LogResponse((apiTimeResult));

            if (apiTimeResult.StatusCode == HttpStatusCode.OK) {
                string apiTimeResultContent = await apiTimeResult.Content.ReadAsStringAsync();
                Dictionary<string, object> values = JsonSerializer.Deserialize<Dictionary<string, object>>(apiTimeResultContent);

                DateTime serverTime = DateTime.Parse(values["serverTime"].ToString());
                Console.WriteLine("Server time: " + serverTime.ToString());

                return serverTime;
            }
            else {
                return this.invalidTime;
            }
        }
        catch (Exception e) {
            ExceptionHandler.LogException(e);
        }

        return this.invalidTime;
    }
}

public static class ExceptionHandler
{
    private static StringBuilder messageBuilder = new StringBuilder();

    public static void LogException(Exception e)
    {
        messageBuilder.AppendLine($"___MESSAGE:{e.Message}.");
        messageBuilder.AppendLine($"___STACKTRACE:{e.StackTrace}.");
        
        Console.WriteLine(messageBuilder.ToString());
    }
}

public static class HttpResponseMessageHandler
{
    private static StringBuilder messageBuilder = new StringBuilder();

    public static async void LogResponse(HttpResponseMessage response)
    {
        string result = await response.Content.ReadAsStringAsync();

        messageBuilder.AppendLine($"___STATUS:{response.StatusCode}.");
        messageBuilder.AppendLine($"___CONTENT:{result}.");

        Console.WriteLine(messageBuilder.ToString());
    }
}

public static class PingReplyHandler
{
    private static StringBuilder messageBuilder = new StringBuilder();

    public static void LogReply(PingReply reply)
    {
        messageBuilder.AppendLine($"__ADDRESS: {reply.Address}");
        messageBuilder.AppendLine($"ROUNTRIP TIME: {reply.RoundtripTime}");
        messageBuilder.AppendLine($"__BUFFER: {reply.Buffer.Length}");
        messageBuilder.AppendLine($"__STATUS: {reply.Status}");

        Console.WriteLine(messageBuilder.ToString());
    }
}

BinanceApp app = new BinanceApp();
app.StartApp();
app.UpdateApp();

__ADDRESS: 13.227.6.143
ROUNTRIP TIME: 8
__BUFFER: 32
__STATUS: Success

___STATUS:OK.
___CONTENT:{}.

___STATUS:OK.
___CONTENT:{}.
___STATUS:OK.
___CONTENT:{"serverTime":1652826843386}.

___MESSAGE:String '1652826843386' was not recognized as a valid DateTime..
___STACKTRACE:   at System.DateTime.Parse(String s)
   at Submission#21.BinanceTimeService.GetServerTime().

